In [1]:
# Warning: Execute this cell only once for the kernel. Reset the kernel if you need changes.
import sys
sys.path.append("../Python")
import ROOT as R
import time
import numpy as np
# Turn jsroot off if you want to make a pdf from this file.
%jsroot on
from root_helpers import SetStyle
from root_helpers import fancy_plot
from root_helpers import print_mc_particle_tree
from root_helpers import print_daughters
# R.EnableImplicitMT()

Welcome to JupyROOT 6.29/01


In [2]:
import os
recompile = True
try:
    if os.path.getmtime('../Python/Utility_Functions_C.so') - os.path.getmtime('../Python/Utility_Functions.C') > 0:
        recompile = False
        print("Recompile is not needed")
    else:
        print("Recompiling: ")
except:
    print("Recompile needed, file not found.")
if recompile:
    R.gROOT.LoadMacro("../Python/Utility_Functions.C++")
else:
    R.gSystem.Load("../Python/Utility_Functions_C.so")
R.Utility_Functions()

Recompiling: 


'Utility Functions V1.0.4 \n'

Info in <TMacOSXSystem::ACLiC>: creating shared library /System/Volumes/Data/data/HPS/Analysis/MC2021/./../Python/Utility_Functions_C.so


Info in <TMacOSXSystem::ACLiC>: creating shared library /System/Volumes/Data/data/HPS/Analysis/MC2021/./../Python/Utility_Functions_C.so


In [3]:
ch = R.TChain("MiniDST")
ch.Add("/data/HPS/data/physrun2021/sim/hpsForward_e-_3.0GeV_z0.0_0_SLIC-v06-00-01_QGSP_BERT_HPS-v2019-3pt7GeV_recon.root")
# ch.Add("/data/HPS/data/physrun2021/pass0/minidst/hps_0147*.root")
print(f"Number of events loaded: {ch.GetEntries()/1e6:7.3f}M")

Number of events loaded:   0.032M


In [4]:
df = R.RDataFrame(ch)
print("Available data names in Tuple:")
ColumnNames=df.GetColumnNames()
ll = 0
pr_colnames = [x for x in ColumnNames if str(x).startswith('')]
for nn in pr_colnames:
    if ll < len(nn):
        ll = len(nn)
for n in range(len(pr_colnames)):
    if n%4 == 0:
        print("")
    print(f"{str(pr_colnames[n]):{ll}s}",end="")

Available data names in Tuple:

ecal_cluster_energy           ecal_cluster_hits             ecal_cluster_nhits            ecal_cluster_seed_energy      
ecal_cluster_seed_index       ecal_cluster_seed_ix          ecal_cluster_seed_iy          ecal_cluster_time             
ecal_cluster_uncor_energy     ecal_cluster_uncor_hits       ecal_cluster_uncor_nhits      ecal_cluster_uncor_seed_energy
ecal_cluster_uncor_seed_index ecal_cluster_uncor_seed_ix    ecal_cluster_uncor_seed_iy    ecal_cluster_uncor_time       
ecal_cluster_uncor_x          ecal_cluster_uncor_y          ecal_cluster_uncor_z          ecal_cluster_x                
ecal_cluster_y                ecal_cluster_z                ecal_hit_energy               ecal_hit_index_x              
ecal_hit_index_y              ecal_hit_time                 event_number                  ext_trigger                   
hodo_cluster_energy           hodo_cluster_ix               hodo_cluster_iy               hodo_cluster_layer            


In [5]:
ones = None
event=108
c_name = "cc"+str(event)
cc1 = R.TCanvas("cc1","Canvas",1000,600)

In [6]:
cc1.Clear()
dfr = df.Range(event,event+1)
# p0 = dfr.Display(("run_number","event_number","ecal_cluster_nhits"))
# p1 = dfr.Display(("ecal_cluster_energy","ecal_cluster_time","ecal_cluster_hits","ecal_cluster_seed_index"))
# p2 = dfr.Display(("ecal_hit_energy","ecal_hit_time","ecal_hit_index_x","ecal_hit_index_y"))
h_ecal = dfr.Histo2D(("h_ecal","Ecal Hits",50,-25.5,24.5,13,-6.5,6.5),"ecal_hit_index_x", "ecal_hit_index_y", "ecal_hit_energy")
# p0.Print()
# p1.Print()
# p2.Print()
h_draw = h_ecal.GetPtr()
ones = fancy_plot(h_ecal, ones, 0x0)
cc1.Draw()
# event +=1

Booking histogram h_ecal_oneslb


In [7]:
R.gSystem.Load("/data/HPS/lib/libMiniDst")

0

In [8]:
# Setup the MiniDST class, which makes for easier event by event data inplection.
# This MiniDST class looks directly into the TTree, so does not use the RDataframe class.
# You need to:
mdst = R.MiniDst()          # Initiate the class
mdst.use_mc_particles=True  # Tell it to look for the MC Particles in the TTree
mdst.use_ecal_cluster_uncor = True;
mdst.use_mc_scoring =True;
mdst.DefineBranchMap()      # Define the map of all the branches to the contents of the TTree
mdst.SetBranchAddressesOnTree(ch) # Connect the TChain (which contains the TTree) to the class.
print(f"MminiDST version = {mdst._version_()}")


MminiDST version = 1.0.8


Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.
Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.
Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.
Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.
Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.


In [9]:
def make_ecal_snaphot(mini_dst, hist, opt=0):
    """Return a 2D histogram for the ECal, with the energy of the hits on the z-axis"""
    if hist is None:
        hist = R.TH2D("hist_ecal","Ecal Hits",50,-25.5,24.5,13,-6.5,6.5)
    else:
        hist.Reset()
    # Fill the histogram by looping over the ECal hits
    for i in range(len(mini_dst.ecal_hit_index_x)):
        hist.Fill(mini_dst.ecal_hit_index_x[i], mini_dst.ecal_hit_index_y[i], mini_dst.ecal_hit_energy[i])
    return hist

In [10]:
cc2 = R.TCanvas("cc2","Canvas",1000,600)
hh = None
ones2 = None
event = 108

In [11]:
ch.GetEntry(event)
hh = make_ecal_snaphot(mdst, hh, 0)
hh.SetStats(0)
cc2.Clear()
ones2 = fancy_plot(hh, ones2, 0x0)
cc2.Update()
cc2.Draw()
event += 1

Booking histogram hist_ecal_oneslb


Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.
Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.
Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.
Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.
Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.


In [25]:
print(f"Run: {mdst.run_number}, Event:{mdst.event_number}")
print(f"NClusters: {len(mdst.ecal_cluster_seed_ix)} :",end="")
for i in range(len(mdst.ecal_cluster_seed_index)):
    print(f" [{mdst.ecal_cluster_seed_index[i]}]({mdst.ecal_cluster_x[i]:7.3f},{mdst.ecal_cluster_y[i]:7.3f},{mdst.ecal_cluster_energy[i]:7.3f})", end=",")
print("\n")
print("   i |   x |   y |  Energy  ")
esum=0
print("----------------------------")
for i in range(len(mdst.ecal_hit_index_x)):
    print(f" {i:3d} | {mdst.ecal_hit_index_x[i]:3d} | {mdst.ecal_hit_index_y[i]:3d} | {mdst.ecal_hit_energy[i]:7.5f} ")
    esum+=mdst.ecal_hit_energy[i]
print(f"Energy sum = {esum}")

Run: 1193700, Event:11
NClusters: 1 : [1](-203.499,-26.262,  2.630),

   i |   x |   y |  Energy  
----------------------------
   0 | -19 |  -2 | 0.02410 
   1 | -18 |  -1 | 1.69546 
   2 | -19 |  -1 | 0.02753 
   3 | -18 |  -2 | 0.06702 
   4 | -17 |  -1 | 0.21466 
   5 | -17 |  -2 | 0.05685 
   6 | -18 |   1 | 0.03121 
Energy sum = 2.11682778224349


In [6]:
event+=1
ch.GetEntry(event)
print_mc_particle_tree(mdst)
primary_index = -1
for i in range(len(mdst.mc_part_z)):
    if abs(mdst.mc_part_z[i])<1e-6:
        primary_index = i
        break
print(f"i_p = {primary_index}")
print(f"Status: {mdst.mc_part_gen_status[primary_index]:d} = {mdst.mc_part_gen_status[primary_index]:032b}")

NameError: name 'mdst' is not defined

In [14]:
ans = R.find_average_end_of_daughters_of_primary_mc_part(mdst.mc_part_z, mdst.mc_part_end_z, mdst.mc_part_daughters)

In [15]:
list(ans)

[1468.774633624398]

In [16]:
h_n_trk = df.Define("n_trk","int n=0; for(int i=0;i<track_type.size();++i){if(track_type[i]==1) n+=1;}; return n").Histo1D(("h_n_trk","N KF tracks",11,-0.5,10.5),"n_trk")

In [17]:
cc3 = R.TCanvas("cc3","Canvas",1000,600)
cc3.SetLogy()
h_n_trk.Draw()
# h_trk_type.Draw()
cc3.Draw()

In [7]:
ch.GetEvent(1002)
print(mdst.track_type)
print(mdst.track_x_at_ecal)
print(mdst.track_y_at_ecal)
print(mdst.ecal_cluster_x)
print(mdst.ecal_cluster_y)
print(mdst.ecal_cluster_z)
print(mdst.ecal_cluster_uncor_x)
print(mdst.ecal_cluster_uncor_y)
print(mdst.ecal_cluster_uncor_z)
print(mdst.ecal_cluster_energy)
print(mdst.ecal_cluster_uncor_energy)


{ 1, 56, 24 }
{ -96.691338, -97.374931, -99.000000 }
{ -66.708488, -66.814224, -99.000000 }
{ -101.53414 }
{ -69.412460 }
{ 1449.6185 }
{ -97.790115 }
{ -70.108475 }
{ 1449.6185 }
{ 3.0117152 }
{ 2.6908541 }


In [8]:
list(R.get_vec_abs(mdst.part.px,mdst.part.py,mdst.part.pz))

[2.981975791233086, 2.948344232063374]

In [9]:
print(mdst.mc_part_end_x)
print(mdst.mc_part_end_y)
print(mdst.mc_part_end_z)
print(mdst.mc_part_energy)
print_mc_particle_tree(mdst)

{ -93.050710, -101.38861, -110.29556, -101.79529, -110.07907 }
{ -80.651426, -69.928371, 135.02972, -70.139109, 133.60789 }
{ 1461.6530, 1450.8160, 1120.3398, 1454.8381, 1121.0679 }
{ 0.00051099889, 2.9999999, 0.0047716166, 0.0000000, 0.0055953349 }


IndexError: index out of range

In [19]:
for i in range(len(mdst.mc_score_type)):
    if mdst.mc_score_type[i]==1 and mdst.mc_score_pz[i]>0.05:
        print(f"[{i:2d}]: {mdst.mc_score_type[i]:1d}, {mdst.mc_score_part_idx[i]}  ({mdst.mc_score_px[i]:7.4f}, {mdst.mc_score_py[i]:7.4f}, {mdst.mc_score_pz[i]:7.4f}), ({mdst.mc_score_x[i]:6.1f}, {mdst.mc_score_y[i]:6.1f}, {mdst.mc_score_z[i]:6.1f})")


[21]: 1, 1  (-0.2830, -0.1418,  2.7367), (-100.6,  -69.5, 1443.0)
[22]: 1, 1  (-0.0169, -0.0081,  0.1677), (-100.4,  -69.3, 1443.0)
[31]: 1, 1  (-0.0054, -0.0028,  0.0528), (-100.6,  -69.6, 1443.0)


In [18]:
list(R.score_plane_hit_energy(mdst.mc_score_type, mdst.mc_score_px, mdst.mc_score_py, mdst.mc_score_pz, mdst.mc_score_x, mdst.mc_score_y, mdst.mc_score_z))

[2.754949414213876, 0.1687648419227899]

In [10]:
mdst.__dir__()

['__module__',
 '__doc__',
 'FULL_CLEAR',
 'Add_Scoring_Plane',
 'Clear',
 'DefineBranchMap',
 'End',
 'GetActiveBranchNames',
 'GetBranchNames',
 'Remove_Scoring_Plane',
 'Run',
 'SetBranchActive',
 'SetBranchAddressesOnTree',
 'SetBranchMap',
 'SetDebugLevel',
 'SetOutputFileName',
 'Start',
 '__assign__',
 '__init__',
 '_version_',
 'branch_map_try_emplace',
 'md_Debug',
 'md_output_file_name',
 'Counter_Freq',
 'md_output_file',
 'md_output_tree',
 'md_abort_tree_fill',
 'use_hodo_raw_hits',
 'use_hodo_hits',
 'use_hodo_clusters',
 'use_ecal_raw_hits',
 'use_ecal_cluster',
 'use_ecal_cluster_uncor',
 'use_ecal_hits',
 'use_svt_raw_hits',
 'use_svt_hits',
 'use_kf_tracks',
 'use_gbl_tracks',
 'use_matched_tracks',
 'use_gbl_kink_data',
 'use_mc_particles',
 'use_mc_scoring',
 'use_kf_particles',
 'use_gbl_particles',
 'particle_types_single',
 'particle_types_double',
 'branch_map',
 'branch_map_active',
 'run_number',
 'event_number',
 'time_stamp',
 'svt_status',
 'trigger',
 'ext

In [12]:
mdst.mc_part_id

vector<int>{ 454904, 454905, 454906, 454907, 454908 }

In [14]:
ch.Print()

******************************************************************************
*Chain   :MiniDST   : /data/HPS/data/physrun2021/sim/hpsForward_e-_3.0GeV_z0.0_0_SLIC-v06-00-01_QGSP_BERT_HPS-v2019-3pt7GeV_recon.root *
******************************************************************************
******************************************************************************
*Tree    :MiniDST   : HPS mini-DST                                           *
*Entries :    31904 : Total =       369429446 bytes  File  Size =  176117565 *
*        :          : Tree compression factor =   2.10                       *
******************************************************************************
*Br    0 :ecal_cluster_energy : vector<double>                               *
*Entries :    31904 : Total  Size=     700207 bytes  File Size  =     250057 *
*Baskets :       10 : Basket Size=     166912 bytes  Compression=   2.80     *
*........................................................................